In [1]:
import pandas as pd
from openpyxl import load_workbook

def matrixf():
    
    ex = pd.read_excel('C:\\Users\\admin\\Documents\\Project Python\\Консалт\\скрипт\\script.xlsm', sheet_name='Sheet1')
    
    # Создаем переменные для вычислений
    file_path = ex['Unnamed: 2'][1]
    bank = ex['Unnamed: 2'][2]
    obligations_type = ex['Unnamed: 2'][3]
    date1='2020-01-01'
    date2='2019-01-01'
    
    # Считываем данные из базы по предоставленному пути
    df = pd.read_csv(file_path, sep=';', encoding = 'cp1251')
    
    # Приводим данные к нужному виду
    df.columns = ['id', 'date', 'delay_day', 'oblig_type', 'bank']
    df.date = pd.to_datetime(df.date, format='%d/%m/%Y')
    
    # Выбираем из датафрейма нужный банк и тип обязательств 
    df = df[df.bank == bank]  
    df = df[df.oblig_type==obligations_type]
    
    # Функция создает категорию на основе периода задолженности
    def delay_category(val):
        if val == 0:
            return 'Без просрочки'
        elif 0 < val <= 30:
            return 'просрочка 0 - 30 дней'
        elif 30 < val <= 60:
            return 'просрочка 31-60 дней'
        elif 60 < val <= 90:
            return 'просрочка 61-90 дней'
        else:
            return 'просрочка 91+ дней'
        
    df['delay_cat'] = df['delay_day'].apply(delay_category)
    
    # Убираем не нужные столбцы    
    df = df.drop(columns=['oblig_type', 'bank', 'delay_day'])
    
    # Делим датафрейм на 2 с разными датами и обьединям по общему id
    df1 = df[df['date'] == date1]
    df2 = df[df['date'] == date2]
    suff1 = '_'+date1
    suff2 = '_'+date2
    df_final = df1.merge(df2, on='id', how='outer', suffixes=(suff1, suff2))

    #Создаем сводную таблицу-матрицу по предоставленному шаблону
    matrix = df_final.pivot_table(index=['date'+suff1, 'delay_cat'+suff1], 
                                  columns=['date'+suff2, 'delay_cat'+suff2], 
                                  values='id', 
                                  aggfunc='count')
    
    # Подключаемся к книге
    book = load_workbook(filename='script.xlsm', read_only=False, keep_vba=True)
    writer = pd.ExcelWriter('script.xlsm', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    
    # Записываем данные на новый лист
    matrix.to_excel(writer, "matrix")
    writer.save()

In [2]:
matrixf()